# Smoothing Test

In [1]:
from sogaPreprocessor import *
from producecfg import *
from libSOGA import *
from libSOGAsmoother import *

torch.set_default_dtype(torch.float64)

### Test 1: Assigning Constants

In [2]:
text = 'y = gauss(0,1); x = 0;'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
output_dist = start_SOGA(cfg)
output_dist

Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1., 0.],
         [0., 0.]]])>

In [3]:
smooth_output_dist = start_SOGA_smooth(cfg)
smooth_output_dist

Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]]])>

In [4]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,y=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,x=0,smoothed:x=0+ gm([1.], [0.], [0.0000100000])>,
 'exit': ExitNode<>}

### Test 2: Assigning Discrete Distributions

In [5]:
text = 'y = gauss(0,1); x = bern(0.5);'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
output_dist = start_SOGA(cfg)
output_dist

Dist<['y', 'x'],pi: tensor([[0.5000],
        [0.5000]])
mu: tensor([[0., 0.],
        [0., 1.]])
sigma: tensor([[[1., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 0.]]])>

In [6]:
#text = 'y = gauss(0,1);x=gm([0.500000,0.500000], [0.0,1.0], [0.0000100000, 0.0000100000]);'
#text = compile2SOGA_text(text)
smooth_output_dist = start_SOGA_smooth(cfg)
smooth_output_dist

Dist<['y', 'x'],pi: tensor([[0.5000],
        [0.5000]])
mu: tensor([[0., 0.],
        [0., 1.]])
sigma: tensor([[[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]]])>

In [7]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,y=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0]),smoothed:x=gm([0.500000,0.500000], [0.0,1.0], [0.0000100000, 0.0000100000])>,
 'exit': ExitNode<>}

### Test 3: Assigning Constant times Discrete

In [8]:
text = 'y = gauss(0,1); x = 2*gm([0.5, 0.5], [0., 1.], [0., 0.]);'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
output_dist = start_SOGA(cfg)
output_dist


Dist<['y', 'x'],pi: tensor([[0.5000],
        [0.5000]])
mu: tensor([[0., 0.],
        [0., 2.]])
sigma: tensor([[[1., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 0.]]])>

In [9]:
smooth_output_dist = start_SOGA_smooth(cfg)
smooth_output_dist

Dist<['y', 'x'],pi: tensor([[0.5000],
        [0.5000]])
mu: tensor([[0., 0.],
        [0., 2.]])
sigma: tensor([[[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 4.0000e-10]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 4.0000e-10]]])>

In [10]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,y=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,x=2*gm([0.5,0.5],[0.,1.],[0.,0.]),smoothed:x=2*gm([0.5,0.5], [0.,1.], [0.0000100000, 0.0000100000])>,
 'exit': ExitNode<>}

### Test 4: Assigning Other Variables

In [11]:
text = 'y = gauss(0,1); x = y;'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
output_dist = start_SOGA(cfg)
output_dist



Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1., 1.],
         [1., 1.]]])>

In [12]:
smooth_output_dist = start_SOGA_smooth(cfg)
smooth_output_dist

Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1.0000, 1.0000],
         [1.0000, 1.0000]]])>

In [13]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,y=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,x=y,smoothed:x=y+ gm([1.], [0.], [0.0000100000])>,
 'exit': ExitNode<>}

### Test 5: If Clauses depending on Extreme Discrete Values

In [14]:
text = 'x = bern(0.5); if x == 1 { x = x + 1; } else { x = x - 1; } end if;'

text=compile2SOGA_text(text)
cfg = produce_cfg_text(text)
#output_dist = start_SOGA(cfg)

#print('Before smoothing')
cfg.node_list


{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0])>,
 'test0': TestNode<test0,x==1>,
 'state1': StateNode<state1,True,x=x+1>,
 'state2': StateNode<state2,False,x=x-1>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [15]:
smooth_output_dist = start_SOGA_smooth(cfg)
print(smooth_output_dist)

cfg.node_list

Dist<['x'],pi: tensor([[5.0000e-01],
        [5.0000e-01],
        [1.4333e-07],
        [1.4333e-07]])
mu: tensor([[ 2.0000e+00],
        [-1.0000e+00],
        [-5.1865e-05],
        [ 5.1865e-05]])
sigma: tensor([[[9.9999e-11]],

        [[1.0000e-10]],

        [[3.2696e-12]],

        [[3.2696e-12]]])>


{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0]),smoothed:x=gm([0.500000,0.500000], [0.0,1.0], [0.0000100000, 0.0000100000])>,
 'test0': TestNode<test0,x==1,smoothed:x > 1 - 0.0000500000 and x < 1 + 0.0000500000>,
 'state1': StateNode<state1,True,x=x+1>,
 'state2': StateNode<state2,False,x=x-1>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

### Test 6: If Clauses depending on Discrete Variables + Double Degenerate update

In [16]:
text = 'x = bern(0.5); if x == 1 { y = 1; } else { y = -1; } end if;'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
#output_dist = start_SOGA(cfg)

#print('Before smoothing')
cfg.node_list


{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0])>,
 'test0': TestNode<test0,x==1>,
 'state1': StateNode<state1,True,y=1>,
 'state2': StateNode<state2,False,y=-1>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [17]:
smooth_output_dist = start_SOGA_smooth(cfg)
print(smooth_output_dist)

cfg.node_list

Dist<['x', 'y'],pi: tensor([[5.0000e-01],
        [5.0000e-01],
        [1.4333e-07],
        [1.4333e-07]])
mu: tensor([[ 1.0000,  1.0000],
        [ 0.0000, -1.0000],
        [ 0.9999, -1.0000],
        [ 1.0001, -1.0000]])
sigma: tensor([[[9.9998e-11, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]],

        [[1.0000e-10, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]],

        [[3.2697e-12, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]],

        [[3.2698e-12, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]]])>


{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0]),smoothed:x=gm([0.500000,0.500000], [0.0,1.0], [0.0000100000, 0.0000100000])>,
 'test0': TestNode<test0,x==1,smoothed:x > 1 - 0.0000500000 and x < 1 + 0.0000500000>,
 'state1': StateNode<state1,True,y=1,smoothed:y=1+ gm([1.], [0.], [0.0000100000])>,
 'state2': StateNode<state2,False,y=-1,smoothed:y=-1+ gm([1.], [0.], [0.0000100000])>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [18]:
cfg.smoothed_vars

['x', 'y', 'y']

### Test 7: If Clauses depending on intermediate values

In [19]:
text = 'x = gm([0.33, 0.33, 0.34], [0., 1. , 2.], [0., 0., 0.]); if x == 1 { y = 1; } else { y = 0; } end if;'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
#output_dist = start_SOGA(cfg)

#print('Before smoothing')
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.33,0.33,0.34],[0.,1.,2.],[0.,0.,0.])>,
 'test0': TestNode<test0,x==1>,
 'state1': StateNode<state1,True,y=1>,
 'state2': StateNode<state2,False,y=0>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [20]:
smooth_output_dist = start_SOGA_smooth(cfg)
print(smooth_output_dist)

cfg.node_list


Dist<['x', 'y'],pi: tensor([[3.3000e-01],
        [3.3000e-01],
        [9.4595e-08],
        [9.4595e-08],
        [3.4000e-01]])
mu: tensor([[1.0000, 1.0000],
        [0.0000, 0.0000],
        [0.9999, 0.0000],
        [1.0001, 0.0000],
        [2.0000, 0.0000]])
sigma: tensor([[[9.9998e-11, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]],

        [[1.0000e-10, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]],

        [[3.2697e-12, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]],

        [[3.2698e-12, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]],

        [[1.0000e-10, 0.0000e+00],
         [0.0000e+00, 1.0000e-10]]])>


{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.33,0.33,0.34],[0.,1.,2.],[0.,0.,0.]),smoothed:x=gm([0.33,0.33,0.34], [0.,1.,2.], [0.0000100000, 0.0000100000, 0.0000100000])>,
 'test0': TestNode<test0,x==1,smoothed:x > 1 - 0.0000500000 and x < 1 + 0.0000500000>,
 'state1': StateNode<state1,True,y=1,smoothed:y=1+ gm([1.], [0.], [0.0000100000])>,
 'state2': StateNode<state2,False,y=0,smoothed:y=0+ gm([1.], [0.], [0.0000100000])>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [21]:
cfg.smoothed_vars

['x', 'y', 'y']

### Test 8: Observing equalities

In [7]:
text = 'x = gauss(0,1); observe(x==1);'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
output_dist = start_SOGA_smooth(cfg)
output_dist

Dist<['x'],pi: tensor([], size=(0, 1))
mu: tensor([], size=(0, 1))
sigma: tensor([], size=(0, 1, 1))>

In [8]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([1.0],[0.000000],[1.000000])>,
 'observe0': ObserveNode<observe0,x==1>,
 'exit': ExitNode<>}

In [3]:
text = 'x = bern(0.5); observe(x==1);'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
output_dist = start_SOGA_smooth(cfg)
output_dist

Dist<['x'],pi: tensor([[1.]])
mu: tensor([[1.]])
sigma: tensor([[[9.9999e-11]]])>

In [4]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0]),smoothed:x=gm([0.500000,0.500000], [0.0,1.0], [0.0000100000, 0.0000100000])>,
 'observe0': ObserveNode<observe0,x==1,smoothed:x > 1 - 0.0000500000 and x < 1 + 0.0000500000>,
 'exit': ExitNode<>}

In [5]:
cfg.smoothed_vars

['x']